In [ ]:
!pip3 install -r requirements.txt

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt

In [2]:
from tensorflow.keras import backend as K


def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

In [ ]:
vcrdci_all_data = pd.read_csv(
    "vcrdci_123_all_data_eg_no_nans.csv",
    skiprows=1,
    names=[
        "media",
        "mos",
        "raw_mos",
        "EgCodecCategory",
        "S-PanSpeed",
        "S-Jiggle",
        "S-FineDetail",
        "S-WhiteLevel",
        "S-BlackLevel",
        "WhiteClipping",
        "S-Blur",
        "viqet-sharpness",
        "NR-IQA-CDI mean",
        "NR-IQA-CDI std",
        "NR-IQA-CDI entropy",
        "NR-IQA-CDI kurtosis",
        "NR-IQA-CDI skewness",
        "bps",
        "eps",
        "mean_error",
        "Ifrac",
        "Pfrac",
        "Bfrac",
        "mean_countP_countI_ratio",
        "bps_pixels",
        "max_relational_error",
        "mean_relational_error",
        "std_raw_vs_all_error",
        "mean_raw_vs_all_error",
        "max_relational_bits",
        "mean_relational_bits",
        "std_raw_vs_all_bits",
        "mean_raw_vs_all_bits",
        "max_relational_countP",
        "mean_relational_countP",
        "std_raw_vs_all_countP",
        "mean_raw_vs_all_countP",
        "mean_error_vs_mean_bits",
        "mean_countP_vs_mean_bits",
        "max_bps",
        "max_to_mean_bits_per_frame",
    ],
)
vcrdci_all_data.pop("media")
vcrdci_all_data.pop("mos")

In [ ]:
vcrdci_60 = vcrdci_all_data[vcrdci_all_data["raw_mos"] >= 60]

In [ ]:
vcrdci_60_small = vcrdci_60.iloc[:500]

In [ ]:
vcrdci_all_features = vcrdci_60_small.copy()
vcrdci_all_labels = vcrdci_all_features.pop("raw_mos")

In [ ]:
categorical_features = ["EgCodecCategory"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder = "passthrough")

transformed_features = transformer.fit_transform(vcrdci_all_features)
vcrdci_all_features = pd.DataFrame(transformed_features)

In [ ]:
feature_train, feature_test, label_train, label_test = train_test_split(vcrdci_all_features, vcrdci_all_labels, test_size = 0.2, random_state = 1)

feature_test.to_csv("feature_test_vcrdci123_small.csv", index=False)

In [ ]:
feature_train = np.array(feature_train)
label_train = np.array(label_train)
feature_test = np.array(feature_test)
label_test = np.array(label_test)

In [ ]:
norm_features = tf.keras.layers.Normalization()
norm_features.adapt(vcrdci_all_features)

In [ ]:
def build_model():
    model = tf.keras.Sequential(
        [
            norm_features,
            tf.keras.layers.Dense(
                500,
                activation="relu",
                input_shape=[len(feature_train[0])],
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(
                500,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=["mse", pearson_r],
    )
    return model

model = build_model()

In [ ]:
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

history = model.fit(
    feature_train,
    label_train,
    epochs=1000,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, PrintDot()],
    verbose=0,
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

val_rmse = np.sqrt(float(hist['val_mse'].min()))
val_pearson = float(hist["val_pearson_r"].min())
print('\nFinal Root Mean Square Error on validation set: {}'.format(round(val_rmse, 3)))
print("Final Pearson Correlation Error on validation set: {}".format(round(val_pearson, 3)))

print("\nEvaluating...")
scores = model.evaluate(feature_test, label_test, verbose=0)
test_loss_mse, test_metric_mse, test_metric_pearson = scores

test_loss_rmse = np.sqrt(test_loss_mse)

print("Loss: RMSE on test set: {}".format(round(test_loss_rmse, 3)))
print("Metric: Pearson on test set: {}".format(round(test_metric_pearson, 3)))

In [ ]:
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Root Mean Square Error [VMAF Score]')
    plt.plot(hist['epoch'], np.sqrt(hist['mse']), label='Train Error')
    plt.plot(hist['epoch'], np.sqrt(hist['val_mse']), label = 'Val Error')
    plt.legend()
    plt.ylim([0,100])
    plt.show()

plot_history()

In [ ]:
model.save("/content/vcrdci123_small_model_test", save_format="tf")
!zip -r vcrdci123_small_model_test.zip vcrdci123_small_model_test/

In [ ]:
feature_test = pd.read_csv("feature_test_vcrdci123_small.csv")
feature_test = np.array(feature_test)

In [ ]:
predictions = model.predict(feature_test)

In [ ]:
with open("tf_predictions.txt", "w+") as f:
  f.write(str(predictions))